In [61]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import string 
import regex as re

from sklearn.model_selection import train_test_split

import io
import spacy

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# df_en_de = pd.read_table('/content/gdrive/MyDrive/deu-eng/deu.txt', names=['eng', 'deu', 'attr'])

In [62]:
df_en_de = pd.read_table('deu-eng/deu.txt', names=['eng', 'deu', 'attr'])

In [63]:
df_en_de = df_en_de.drop('attr',axis = 1).rename(columns = {'eng':'english', 'deu':'german'})

In [64]:
df_en_de

,english,german
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!
...,...,...
251715,If someone who doesn't know your background sa...,"Wenn jemand Fremdes dir sagt, dass du dich wie..."
251716,If someone who doesn't know your background sa...,"Wenn jemand, der nicht weiß, woher man kommt, ..."
251717,It may be impossible to get a completely error...,"Es ist wohl unmöglich, einen vollkommen fehler..."
251718,I know that adding sentences only in your nati...,"Ich weiß wohl, dass das ausschließliche Beitra..."


In [65]:
# Lowercase all characters
df_en_de['english'] = df_en_de['english'].apply(lambda x: x.lower())
df_en_de['german'] = df_en_de['german'].apply(lambda x: x.lower())

# Remove quotes
df_en_de['english'] = df_en_de['english'].apply(lambda x: re.sub("'", '', x))
df_en_de['german'] = df_en_de['german'].apply(lambda x: re.sub("'", '', x))

# Set of all special characters
exclude = set(string.punctuation) 

# Remove all the special characters
df_en_de['english'] = df_en_de['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df_en_de['german']=df_en_de['german'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Add start and end tokens to target sequences
df_en_de['german'] = df_en_de['german'].apply(lambda x : 'START_ '+ x + ' _END')



In [66]:
df_en_de.head()

,english,german
0,go,START_ geh _END
1,hi,START_ hallo _END
2,hi,START_ grüß gott _END
3,run,START_ lauf _END
4,run,START_ lauf _END


In [67]:
#rename dataframe for convenience
pairs = df_en_de

In [68]:
pairs['english_length'] = pairs['english'].apply(lambda x: len(x.split()))
pairs['german_length'] = pairs['german'].apply(lambda x: len(x.split()))
max_len = 20
pairs = pairs[pairs['english_length'] <= max_len]
pairs = pairs[pairs['german_length'] <= max_len]
pairs

,english,german,english_length,german_length
0,go,START_ geh _END,1,3
1,hi,START_ hallo _END,1,3
2,hi,START_ grüß gott _END,1,4
3,run,START_ lauf _END,1,3
4,run,START_ lauf _END,1,3
...,...,...,...,...
251567,charles moore created forth in an attempt to i...,START_ charles moore entwickelte forth in eine...,15,19
251568,every student who has graduated from our unive...,START_ jeder student mit einem abschluss von u...,20,18
251570,its dangerous to assume that all of the senten...,START_ es ist eine gefährliche annahme dass al...,20,18
251587,the color green is often associated with money...,START_ die farbe grün wird oft mit geld und de...,20,20


In [146]:
# Vocabulary of English
all_en_words=set()
for eng in pairs['english']:
    for word in eng.split():
        if word not in all_en_words:
            all_en_words.add(word)

# Vocabulary of German 
all_de_words=set()
for de in pairs['german']:
    for word in de.split():
        if word not in all_de_words:
            all_de_words.add(word)

# Max Length of source sequence
length_list=[]
for l in pairs['english']:
    length_list.append(len(l.split(' ')))
max_length_src = np.max(length_list)

# Max Length of target sequence
length_list=[]
for l in pairs['german']:
    length_list.append(len(l.split(' ')))
max_length_tar = np.max(length_list)

input_words = sorted(list(all_en_words))
target_words = sorted(list(all_de_words))

# Calculate Vocab size for both source and target
num_encoder_tokens = len(all_en_words)
num_decoder_tokens = len(all_de_words)

#""" find out why you add 1"""
#num_decoder_tokens += 1 # For zero padding 

# Create word to token dictionary for both source and target
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

# Create token to word dictionary for both source and target
reverse_input_token_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_token_index = dict((i, word) for word, i in target_token_index.items())

In [147]:
target_token_index['START_'], target_token_index['_END']

(367, 368)

In [148]:
max_length_src, max_length_tar, num_decoder_tokens

(20, 20, 37773)

In [149]:
pairs['english']

0                                                        go
1                                                        hi
2                                                        hi
3                                                       run
4                                                       run
                                ...                        
251567    charles moore created forth in an attempt to i...
251568    every student who has graduated from our unive...
251570    its dangerous to assume that all of the senten...
251587    the color green is often associated with money...
251632    a great relationship is based on two main prin...
Name: english, Length: 251049, dtype: object

In [150]:
# Try with a small dataset first, due to problems with dealing with too large a batch at a time
X_train, X_test, y_train, y_test = train_test_split(pairs['english'], pairs['german'], test_size=0.99, random_state=101)

In [151]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [152]:
X_train

array(['he is my classmate', 'hes been asleep for ten hours now',
       'i dont know whos involved', ...,
       'we must go there whether we like it or not',
       'who would look after my children if i died',
       'i should probably lose a few pounds'], dtype=object)

In [153]:
X_train.shape

(2510,)

In [154]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):          # j = batch number
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')

            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            decoder_target_data = np.transpose(decoder_target_data, axes = [1, 0, 2])
            decoder_target_data = list(decoder_target_data)
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [173]:
batch = generate_batch()
[encoder_input_data, decoder_input_data], decoder_target_data = next(batch)

In [181]:
type(encoder_input_data)
type(decoder_input_data)
type(decoder_target_data)
decoder_target_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [156]:
decoder_target_data[0].shape

(128, 37773)

<h3> Model for training </h3>

In [157]:
Tx = max_len
Ty = Tx

# Modify these later based on spacy's word vectors
input_embedding_dims = 100
output_embedding_dims = 100

n_a = 64

In [186]:
# layers as global variables

# Create layer objects LSTM_cell and densor
decoder_LSTM_cell = LSTM (n_a, return_state = True)        
densor = Dense(num_decoder_tokens, activation='softmax')

# Create reshaper object – will be used in function "learning_model"
reshaper = Reshape((1, output_embedding_dims))

<h3> Training Model </h3>

In [203]:
aaa = Input(shape = (None,))
aaa.shape

TensorShape([None, None])

In [206]:
### Encoder part ###

# Input layer for encoder (English)
encoder_inputs = Input(shape=(Tx,), dtype = 'int32')            # (None, Tx) -- sequence of integers

# Embedding layer for encoder (English)
enc_emb_layer = Embedding (input_dim = num_encoder_tokens + 1, output_dim = input_embedding_dims, mask_zero = True)

# Turn input sequence (English) into embedding vectors
encoder_embeddings = enc_emb_layer(encoder_inputs)                 # (None, Tx, input_embedding_dims)

# LSTM layer for encoder (English)
encoder_lstm = LSTM(units = n_a, return_state = True, name = 'encoder_LSTM')           # Note that this LSTM layer computes on ALL Tx values at once.

# Pass input embedding vectors (English) through encoder LSTM
encoder_outputs, a, c = encoder_lstm(encoder_embeddings)

# Save state vectors from encoder LSTM
encoder_states = [a,c]


### Decoder part ###

# Input layer for decoder (German)
decoder_inputs = Input(shape = (Ty,), dtype = 'int32')      # (None, Ty)

# Embedding layer for decoder (German)
dec_emb_layer = Embedding(input_dim = num_decoder_tokens + 1, output_dim = output_embedding_dims, mask_zero = True)

# Turn decoder input sequence (German) into embedding vectors
decoder_embeddings = dec_emb_layer(decoder_inputs)  # (None, Ty, output_embedding_dims)

# Initialise list of outputs
decoder_outputs = []

# Loop over each time-step of decoder input (German) 
for t in range (Ty):
    # Select embedding vector for time-step t
    dec_emb_t = decoder_embeddings[:,t,:]       # (None, output_embedding_dims)

    # Reshape embedding vector for time-step t       
    dec_emb_t = reshaper(dec_emb_t)             # (None, 1, output_embedding_dims)

    # Get a and c for time-step t from decoder LSTM 
    a, _, c = decoder_LSTM_cell(inputs = dec_emb_t, initial_state = encoder_states)
    encoder_states = [a,c]

    # Pass a for time-step t from decoder LSTM through Dense layer
    out = densor(a)
    decoder_outputs.append(out)

# encoder_inputs is numpy array
# decoder_inputs is numpy array
# decoder_outputs is list of numpy arrays (one-hot vectors)
model = Model (inputs = [encoder_inputs, decoder_inputs], outputs = decoder_outputs)

<h3> Inference Model </h3>

In [219]:
#### Get a and c vectors from encoder ####

# Create embedding vectors for encoder input sequence (English)
                                                                     # encoder_inputs = (None, Tx)
encoder_embeddings_2 = enc_emb_layer(encoder_inputs)                 # (None, Tx, input_embedding_dims)

# Pass embedding vectors through encoder LSTM
encoder_outputs_2, a_2, c_2 = encoder_lstm(encoder_embeddings_2)     # Use same encoding LSTM layer

# Store a and c vectors from encoder
encoder_states_2 = [a_2,c_2]                                         

# Create model. 
# inputs = encoder input sequence; outputs = a and c vectors
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states_2)          #encoder_inputs = (None, Tx)
                                                                                    # encoder_states_2 = [a_2, c_2], a_2 & c_2 are numpy arrays        


### Use a and c vectors from encoder to generate prediction ###

# Input layers for a0 and c0 vectors into decoder
#decoder_input2 = Input(shape = (None,))
decoder_state_input_a = Input(shape = (n_a))                # (None, n_a)
decoder_state_input_c = Input(shape = (n_a))                # (None, n_a)
decoder_states_inputs = [decoder_state_input_a, decoder_state_input_c]

# Convert decoder input sequence (German) into embedding vectors
                                                                # decoder_inputs = (None, Ty)
decoder_embeddings_2 = dec_emb_layer(decoder_inputs)            # (None, Ty, output_embedding_dims)

# Select embedding vector for time-step 0 (START_ token)
decoder_embeddings_2 = decoder_embeddings_2[:,0,:]              # (None, output_embedding_dims)
# Reshape into correct dimensions
decoder_embeddings_2 = reshaper(decoder_embeddings_2)           # (None, 1, output_embedding_dims)

outputs2 = []

# look over all time-steps in the decoder input sequence
# number of time_steps will increase by 1 each time we make a prediction later
for t in range(decoder_inputs.shape[1]):                 

    # pass in a, c, and decoder input vectors (time-step t-1) into decoder LSTM
    # Note, inputs (for decoder_LSTM_cell) = (None, 1, output_embedding_dims)
    # Get updated a, c vectors
    decoder_state_input_a, _, decoder_state_input_c = decoder_LSTM_cell(inputs = decoder_embeddings_2, initial_state = decoder_states_inputs)
    decoder_state_inputs = [decoder_state_input_a, decoder_state_input_c]

    # Pass "a" vector through Dense layer
    out = densor(decoder_state_input_a)                  # out = (None, num_decoder_tokens) 

    
    # Find index for prediction
    max_idx = tf.math.argmax(out, -1)                       # max_idx = (None, 1)
    # Convert prediction into embedding vector
    decoder_embeddings_2 = dec_emb_layer(max_idx+1)           # decoder_embeddings_2 = (None, output_embedding_dims)

    # Reshape embedding vector
    decoder_embeddings_2 = reshaper(decoder_embeddings_2)   # (None, 1, output_embedding_dims)

# decoder_inputs = numpy array
# decoder_states_inputs = list of numpy arrays
# [decoder_inputs] + decoder+states_inputs = list of numpy arrays
decoder_model = Model(inputs = [decoder_inputs]+ decoder_states_inputs, 
                    outputs = out                           # "out" = softmax output for final time-step inspected
                                                            # out = (None, num_decoder_tokens)
                    )                                           


In [182]:
list1 = [1,2,3]
list2 = [10, 11]
list1 + list2

[1, 2, 3, 10, 11]

In [196]:
ar1 = np.zeros((1,1))
ar1 = np.append(ar1, 1)
ar1 = np.append(ar1,2)

In [200]:
ar1.reshape(1,-1).shape

(1, 3)

In [228]:
reverse_target_token_index[3999]

'befüllte'

In [236]:
def decode_sequence(input_seq):
    """
    input_seq = numpy array of sequences (1,Ty)
    """

    # Encode the input as state vectors.
    states_values = encoder_model.predict(input_seq)         # input_seq = (1,Ty)
                                                             # states_values = [a, c] from encoder, where a,c are numpy arrays
    

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,Ty))                            
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']          # One integer

    
    
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    j = 1
    while not stop_condition and j <Ty:
        print(target_seq.shape, target_seq)
        output_tokens = decoder_model.predict([target_seq] + states_values)     
        # inputs = [decoder_inputs]+ decoder_states_inputs = list of three numpy arrays

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[:, 1])
        sampled_char = reverse_target_token_index[sampled_token_index + 1]
        decoded_sentence += ' '+sampled_char
        print(decoded_sentence, len(decoded_sentence))
        
        # Exit condition: either hit max length or find stop token.
        if (sampled_char == '_END' or len(decoded_sentence.split()) > Ty):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq[0,j] = sampled_token_index+1
        
        j+=1
        print(j)
        
    
    return decoded_sentence


In [190]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [191]:
# encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states_2)

epochs = 1

#(encoder_input_data, decoder_input_data), decoder_target_data = generate_batch()
history = model.fit(generate_batch(), steps_per_epoch=X_train.shape[0]//128, epochs = epochs, verbose = 1)

19/19 [==============================] - 39s 618ms/step - loss: 58.5120 - dense_8_loss: 7.3553 - dense_8_1_loss: 8.4977 - dense_8_2_loss: 7.9895 - dense_8_3_loss: 8.3832 - dense_8_4_loss: 7.5750 - dense_8_5_loss: 6.0467 - dense_8_6_loss: 4.3263 - dense_8_7_loss: 2.9849 - dense_8_8_loss: 1.9643 - dense_8_9_loss: 1.2915 - dense_8_10_loss: 0.8003 - dense_8_11_loss: 0.4746 - dense_8_12_loss: 0.2945 - dense_8_13_loss: 0.2207 - dense_8_14_loss: 0.1595 - dense_8_15_loss: 0.0720 - dense_8_16_loss: 0.0466 - dense_8_17_loss: 0.0213 - dense_8_18_loss: 0.0083 - dense_8_19_loss: 0.0000e+00 - dense_8_accuracy: 0.0082 - dense_8_1_accuracy: 0.0012 - dense_8_2_accuracy: 0.0082 - dense_8_3_accuracy: 0.0600 - dense_8_4_accuracy: 0.1501 - dense_8_5_accuracy: 0.1933 - dense_8_6_accuracy: 0.1678 - dense_8_7_accuracy: 0.1209 - dense_8_8_accuracy: 0.0847 - dense_8_9_accuracy: 0.0580 - dense_8_10_accuracy: 0.0432 - dense_8_11_accuracy: 0.0230 - dense_8_12_accuracy: 0.0123 - dense_8_13_accuracy: 0.0070 - dense_

In [232]:
inpt_sq = X_train[0].split()
inpt_sq = [input_token_index[word] for word in inpt_sq]
#inpt_sq = np.array(inpt_sq).reshape(1,5)
#inpt_sq.shape

max_len = 20
input_data = np.zeros((1,max_len))
for i, num in enumerate(inpt_sq):
  input_data[0,i] = num
input_data.shape

(1, 20)

In [237]:
decode_sequence(input_data)

(1, 20) [[367.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
 0 2
2
(1, 20) [[367.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
 0 0 4
3
(1, 20) [[367.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
 0 0 0 6
4
(1, 20) [[367.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
 0 0 0 0 8
5
(1, 20) [[367.   1.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
 0 0 0 0 0 10
6
(1, 20) [[367.   1.   1.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
 0 0 0 0 0 0 12
7
(1, 20) [[367.   1.   1.   1.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
 0 0 0 0 0 0 0 14
8
(1, 20) [[367.   1.   1.   1.   1.   1.   1.   1.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.  

' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0'